In [1]:
import sys
sys.path.append("C:\\Users\\79165\\anaconda3\\envs\\slovnik\\lib\\site-packages\\")

sys.path.append("C://dev/ISV_data_gathering/")
sys.path.append("C://dev//ISV_NLP_utils//isv_nlp_utils/")



In [2]:
from isv_nlp_utils.flavorization.parsing import parse_multireplacer_rules
from isv_nlp_utils.flavorization.tokenizer import compute_annotated_tokens, pretty_stringify
from isv_nlp_utils.flavorization.replacer import process_multireplacing, morphological_flavorise


In [3]:
from isv_nlp_utils.flavorization.selector import produce_string, filter_good_spellings, filter_lingua, init_detector, init_hunspell


In [4]:

from isv_nlp_utils import constants
from isv_nlp_utils.slovnik import get_slovnik
# from isv_translate import translate_sentence, postprocess_translation_details, prepare_parsing

from ast import literal_eval
import os
import glob


In [5]:
from razdel import sentenize


In [6]:
# from .selector import select_randomly, select_lingua

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
slovnik = get_slovnik()
slovnik = slovnik['words']

morph = constants.create_etm_analyzer(r"C:\dev\ISV_data_gathering\\")


Found 'slovnik.pkl' file, using it


In [9]:
Src = (
    "každy ščęstlivy pės uměje graciozno padati v ćuđų, hlådnų vodų: vzęti rybu fugu iz vȯzduha. Hćų prěporųčiti: ględi pěše Troicky most v grådu Čeljabinsku, žędam foto za ženų. "
    "Izměnil byh nemnogo temų, zatože za nas vsih imajų taky maly svętočno-novolětny dar. Jedna gospođa profesorica iz mojego universiteta, to znači Universiteta Adama Mickiewicza v Poznaǌu, poslala mi je někake teksty, sråvǌajųće porędȯk slov (sintaksis) slovjańskyh językov. Možno to bųde koristno k daĺšej universalizacĳi MS? Govorjų uže tutčas, že ne jesm na to voobće ględěl. Teksty sųt na češskom, russkom i francuzskom, ako dobro pamętajų. Prĳętno čitańje! "
    "V něktoryh slučajah jest potrěbno pogleděti na druge jezyky, da by uznati optimalnu metodu skazati něčto. "
    "Takože možno prěględěti naš fraznik, tam jest detaljna informacija o několikyh desetkah najvyše važnyh slov. "
)


In [10]:
rules_struct = {}
flavor_rules = {}
for LANG in ['ru', 'cs']:
    rules_struct[LANG] = parse_multireplacer_rules(
        r"C:\dev\razumlivost\src\flavorizers\{}.ts".format(LANG)
    )
    with open(r"C:\dev\razumlivost\src\flavorizers\morpho_{}.txt".format(LANG), "r", encoding="utf8") as f:
        flavor_rules[LANG] = literal_eval(f.read())



In [11]:
tokens_base = compute_annotated_tokens(Src, morph, slovnik)

# for i in range(len(tokens)):
#     if tokens[i].text == ['novolětny']:
#         print(i)

In [12]:
hunspell_names = {"ru": "ru_RU", "cs": "cs_CZ"}
iso_names = {"ru": "rus", "cs": "ces"}
ietf_bcp = {"ru": "rus_Cyrl", "cs": "ces_Latn"}

In [13]:
flavorized_result = {}

for LANG in ['ru', 'cs']:

    tokens = morphological_flavorise(tokens_base, morph, flavor_rules[LANG], debug_indices=[99, 71, 47, 21, 1])
    tokens = process_multireplacing(tokens, rules_struct[LANG], debug_indices=[99, 71, 47, 21, 1])
    flavorized_result[LANG + "_raw"] = pretty_stringify(tokens)
    
    hlang = hunspell_names[LANG]
    ilang = iso_names[LANG]
    h = init_hunspell(hlang)

    flavorized = produce_string(tokens, hlang, ilang)
    flavorized_result[LANG] = flavorized
    flavorized_result[LANG + "_sent"] = list(sentenize(flavorized))


ščęstlivy ščęstlivyj
'ORO-OLO'
['hlådnuju']  =>  ['holodnuju', 'hladnuju']
'ORO-OLO'
['grådu']  =>  ['gradu', 'gorodu']
'ORO-OLO'
['sråvnjajųće']  =>  ['sravnjajųće', 'sorovnjajųće']
'Č-ŠČ'
['ćuđuju']  =>  ['čuđuju', 'ščuđuju']
'Č-ŠČ'
['hćų']  =>  ['hščų', 'hčų']
'Č-ŠČ'
['sravnjajųće', 'sorovnjajųće']  =>  ['sravnjajųče', 'sravnjajųšče', 'sorovnjajųče', 'sorovnjajųšče']
'Č-ŠČ'
['vȯobće']  =>  ['vȯobšče', 'vȯobče']
'ŽD'
['čuđuju', 'ščuđuju']  =>  ['čudžuju', 'ščužuju', 'ščudžuju', 'čužuju', 'čužduju', 'ščužduju']
'ŽD'
['gospođa']  =>  ['gospoža', 'gospožda', 'gospodža']
'Little Yus (Non-J)'
['ščęstlivyj']  =>  ['ščьastlivyj', 'ščеstlivyj']
'Little Yus (Non-J)'
['vzętь']  =>  ['vzеtь', 'vzьatь']
'Little Yus (Non-J)'
['ględi']  =>  ['glеdi', 'glьadi']
'Little Yus (Non-J)'
['žędajų']  =>  ['žьadajų', 'žеdajų']
'Little Yus (Non-J)'
['svętočno-novolětnyj']  =>  ['svеtočno-novolětnyj', 'svьatočno-novolětnyj']
'Little Yus (Non-J)'
['porędȯk']  =>  ['porеdȯk', 'porьadȯk']
'Little Yus (Non-J)'
[

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


In [17]:


def translate(text, src_lang, tgt_lang, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    results = []
    for sent in sentenize(text):
        inputs = tokenizer(sent.text, return_tensors='pt')
        result = model.generate(
            **inputs.to(model.device), 
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            **kwargs
        )
        results.append(
            tokenizer.decode(result[0], skip_special_tokens=True)
        )
    return results


In [18]:
translation = {}
for LANG in ['cs', 'ru']:
    l = ietf_bcp[LANG]
    translation[LANG] = translate(flavorized_result[LANG], l, l)
    translation

C:\Users\79165\anaconda3\envs\repdexai\lib\site-packages\transformers\generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [19]:
from sentence_transformers import SentenceTransformer
#from scipy.spatial import distance
from scipy.spatial.distance import pdist

model_sim = SentenceTransformer('sentence-transformers/LaBSE')


In [20]:
flavorized_result[LANG] = list(sentenize(flavorized_result[LANG]))

In [21]:
for i, sent1 in enumerate(sentenize(Src)):
    
    sents = [sent1.text] + [translation[LANG][i] for LANG in ['cs', 'ru']]
    vectors = [model_sim.encode(s) for s in sents]
    
    sim = pdist(vectors, "cosine")

    print("----")
    print(sim)
    print(sent1.text)
    print()
    for LANG in ['cs', 'ru']:
        print(translation[LANG][i])
        print("+++ " + flavorized_result[LANG + "_sent"][i].text + " +++")
    print()


----
[0.31651057 0.33739552 0.15926062]
každy ščęstlivy pės uměje graciozno padati v ćuđų, hlådnų vodų: vzęti rybu fugu iz vȯzduha.

Každý šťastný pes umí hravě spadat do hladové vody: chytit rybí fugu s vzduchem.
+++ každy štěstlivy pes uměje hraciozné padat v cidjou, hladnou vodu: vzat rybu fugu se vzduhe. +++
Каждый щедрый пес умеет грациозно падать в чужую, холодную воду: вытаскивать рыбу фугу из воздуха.
+++ каждый щестливый пес умеет грациозно падать в чуждую, холодную воду: взять рыбу фугу из воздуха. +++

----
[0.37078422 0.24127149 0.38819901]
Hćų prěporųčiti: ględi pěše Troicky most v grådu Čeljabinsku, žędam foto za ženų.

Připomínám vám, abyste si prohlédli Troickou most v Čelabinské hradě.
+++ Hcem přeporoučit: hladi pěše Troicky most v gradu Čelabinskou, žedam foto za ženou. +++
Хочу переполнить поручение: посмотреть на Троицкий мост в Челябинске, хочу фотографию жены.
+++ Хщу перепоручить: гляди пеше Троицкы мост в городу Челябинску, жадаю фото за жену. +++

----
[0.4386